# Area under the receiver operating curve

In [ ]:
# standard library
from typing import Tuple

In [ ]:
# numerical
import numpy as np
import pandas as pd

from scipy.stats import gaussian_kde
from scipy.integrate import trapezoid
from scipy.stats import multivariate_normal

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
# plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
# local
import sys
sys.path.append('../lib')

import compstats

In [ ]:
# worksheet
from IPython.core.pylabtools import figsize
figsize(11, 5)

In [ ]:
# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

### Area under ROC

As a way of understanding AUC ROC, let's look at the relationship between AUC and Cohen's effect size.

Cohen's effect size, `d`, expresses the difference between two groups as the number of standard deviations between the means.

As `d` increases, we expect it to be easier to distinguish between groups, so we expect AUC to increase.

I'll start in one dimension and then generalize to multiple dimensions.a

Here are the means and standard deviations for two hypothetical groups.

In [ ]:
mu1 = 0
sigma = 1

d = 1
mu2 = mu1 + d;

I'll generate two random samples with these parameters.

In [ ]:
n = 1000
sample1 = np.random.normal(mu1, sigma, n)
sample2 = np.random.normal(mu2, sigma, n);

If we put a threshold at the midpoint between the means, we can compute the fraction of Group 0 that would be above the threshold.

I'll call that the false positive rate.

In [ ]:
thresh = (mu1 + mu2) / 2
np.mean(sample1 > thresh)

And here's the fraction of Group 1 that would be below the threshold, which I'll call the false negative rate.

In [ ]:
np.mean(sample2 < thresh)

### Plotting misclassification

To see what these overlapping distributions look like, I'll plot a kernel density estimate (KDE).

In [ ]:
def make_kde(sample: np.ndarray) -> pd.Series:
    """Kernel density estimate.
    
    sample: sequence
    
    returns: Series
    """
    xs = np.linspace(-4, 4, 101)
    kde = gaussian_kde(sample)
    ys = kde.evaluate(xs)
    return pd.Series(ys, index=xs)

In [ ]:
def plot_kde(kde: pd.Series, clipped: pd.Series, color: str):
    """Plot a KDE and fill under the clipped part.
    
    kde: Series
    clipped: Series
    color: string
    """
    plt.plot(kde.index, kde, color=color)
    plt.fill_between(clipped.index, clipped, color=color, alpha=0.3)

In [ ]:
def plot_misclassification(sample1: np.array, sample2: np.array, thresh: float):
    """Plot KDEs and shade the areas of misclassification.
    
    sample1: sequence
    sample2: sequence
    thresh: number
    """
    kde1 = make_kde(sample1)
    # sample1 lower. shade above the threshold (false positives)
    clipped = kde1[kde1.index>=thresh]
    plot_kde(kde1, clipped, 'C0')

    kde2 = make_kde(sample2)
    # sample2 higher. shade below the threshold (false negatives)
    clipped = kde2[kde2.index<=thresh]
    plot_kde(kde2, clipped, 'C1')

Here's what it looks like with the threshold at 0.  There are many false positives, shown in blue, and few false negatives, in orange.

In [ ]:
plot_misclassification(sample1, sample2, 0)

With a higher threshold, we get fewer false positives, at the cost of more false negatives.

In [ ]:
plot_misclassification(sample1, sample2, 1)

### The receiver operating curve

The receiver operating curve (ROC) represents this tradeoff.

To plot the ROC, we have to compute the false positive rate (which we saw in the figure above), and the true positive rate (not shown in the figure).

The following function computes these metrics.

In [ ]:
def fpr_tpr(sample1: np.ndarray, sample2: np.ndarray, thresh: float) -> Tuple[np.float64, np.float64]:
    """Compute false positive and true positive rates.
    
    sample1: sequence
    sample2: sequence
    thresh: number
    
    returns: tuple of (fpr, tpf)
    """
    # lower sample above the threshold
    fpr = np.mean(sample1>thresh)
    # true positives - upper sample to the right of the threshold
    tpr = np.mean(sample2>thresh)
    return fpr, tpr

When the threshold is high, the false positive rate is low, but so is the true positive rate.

In [ ]:
fpr_tpr(sample1, sample2, 1)

As we decrease the threshold, the true positive rate increases, but so does the false positive rate.

In [ ]:
fpr_tpr(sample1, sample2, 0)

The ROC shows this tradeoff over a range of thresholds.

I sweep thresholds from high to low so the ROC goes from left to right.

In [ ]:
def plot_roc(sample1: np.array, sample2: np.array, label: str):
    """Plot the ROC curve and return the AUC.
    
    sample1: sequence
    sample2: sequence
    label: string
    
    returns: AUC
    """
    threshes = np.linspace(5, -3)
    # a list of (fpr, tpr) tuples
    roc = [
        fpr_tpr(sample1, sample2, thresh) for thresh in threshes
    ]

    fpr, tpr = np.transpose(roc)
    plt.plot(fpr, tpr, label=label)
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    
    auc = trapezoid(tpr, fpr)
    return auc

Here's the ROC for the samples.

With `d=1`, the area under the curve is about 0.75.  That might be a good number to remember.

In [ ]:
auc = plot_roc(sample1, sample2, '')

Now let's see what that looks like for a range of `d`.

In [ ]:
mu1 = 0
sigma = 1
n = 1000

res = []
# differences between mu2 and mu1 - i.e the effect size
for mu2 in [3, 2, 1.5, 0.75, 0.25]:
    sample1 = np.random.normal(mu1, sigma, n)
    sample2 = np.random.normal(mu2, sigma, n)
    d = (mu2-mu1) / sigma
    label = f'd = {d:0.2g}'
    auc = plot_roc(sample1, sample2, label)
    res.append((d, auc))
    
plt.legend();

In [ ]:
np.round(res, 2)

This function computes AUC as a function of `d`.

In [ ]:
def plot_auc_vs_d(res: np.array, label=None):
    d, auc = np.transpose(res)
    plt.plot(d, auc, alpha=0.8, label=label)
    plt.xlabel('Cohen effect size')
    plt.ylabel('Area under ROC')

The following figure shows AUC as a function of `d`.

In [ ]:
plot_auc_vs_d(res)

Not suprisingly, AUC increases as `d` increases.

### Multivariate distributions

Now let's see what happens if we have more than one variable, with a difference in means along more than one dimension.

First, I'll generate a 2-D sample with `d=1` along both dimensions.

In [ ]:
d = 1
mu1 = [0, 0]
mu2 = [d, d]

rho = 0
sigma = [[1, rho], [rho, 1]]

In [ ]:
# 1000x2
sample1 = multivariate_normal(mu1, sigma).rvs(n)
sample2 = multivariate_normal(mu2, sigma).rvs(n);

The mean of `sample1` should be near 0 for both features.

In [ ]:
# column means
np.mean(sample1, axis=0)

And the mean of `sample2` should be near 1.

In [ ]:
np.mean(sample2, axis=0)

The following scatterplot shows what this looks like in 2-D.

In [ ]:
figsize(8, 8)
# 1000x2 -> 2x1000
# the '.' means a scatter plot
plt.plot(*sample1.transpose(), '.', alpha=0.3, color='C0')
plt.plot(*sample2.transpose(), '.', alpha=0.3, color='C1')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Scatter plot for samples with d=1 in both dimensions');
plt.xlim((-3, 4))
plt.ylim((-3, 4));

Some points are clearly classifiable, but there is substantial overlap in the distributions.



We can see the same thing if we estimate a 2-D density function and make a contour plot.

In [ ]:
np.linspace(-2, 2).shape

In [ ]:
np.array(np.meshgrid(np.linspace(-2, 2), np.linspace(-2, 2))).shape

In [ ]:
# Based on an example at https://plot.ly/ipython-notebooks/2d-kernel-density-distributions/

def kde_scipy(sample: np.ndarray):
    """Use KDE to compute an array of densities.
    
    sample: sequence
    
    returns: tuple of matrixes, (X, Y, Z)
    """
    x = np.linspace(-4, 4)
    y = x
    X, Y = np.meshgrid(x, y)
    # ravel converts 100x100 grids into 10000x1 flattened array
    positions = np.vstack([Y.ravel(), X.ravel()])

    kde = gaussian_kde(sample.T)
    kde(positions)
    # make z into a 100x100 2d matrix
    Z = np.reshape(kde(positions).T, X.shape)

    return [X, Y, Z]

In [ ]:
figsize(8, 8)
X, Y, Z = kde_scipy(sample1) 
plt.contour(X, Y, Z, cmap=plt.cm.Blues, alpha=0.7)

X, Y, Z = kde_scipy(sample2) 
plt.contour(X, Y, Z, cmap=plt.cm.Oranges, alpha=0.7)

plt.xlabel('X')
plt.ylabel('Y')
plt.title('KDE for samples with d=1 in both dimensions');

### Classification with logistic regression

To see how distinguishable the samples are, I'll use logistic regression.

To get the data into the right shape, I'll make two DataFrames, label them, concatenate them, and then extract the labels and the features. 

In [ ]:
pd.DataFrame(sample1)

In [ ]:
df1 = pd.DataFrame(sample1)
# sample 1
df1['label'] = 1
df1.describe()

In [ ]:
df1[[0,1]].corr()

In [ ]:
df2 = pd.DataFrame(sample2)
# sample 2
df2['label'] = 2
df2.describe()

In [ ]:
df2[[0,1]].corr()

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
df.label.value_counts()

In [ ]:
df.head()

`X` is the array of features; `y` is the vector of labels.

In [ ]:
X = df[[0, 1]]
y = df.label;

Now we can fit the model.

In [ ]:
model = LogisticRegression(solver='lbfgs').fit(X, y);

And compute the AUC.

In [ ]:
y_pred_prob = model.predict_proba(X)[:,1]
auc = roc_auc_score(y, y_pred_prob)

With two features, we can do better than with just one.



### AUC as a function of rho

The following function contains the code from the previous section, with `rho` as a parameter.

In [ ]:
def multivariate_normal_auc(d, rho=0):
    """Generate multivariate normal samples and classify them.
    
    d: Cohen's effect size along each dimension
    num_dims: number of dimensions
    
    returns: AUC
    """
    mu1 = [0, 0]
    mu2 = [d, d]

    sigma = [[1, rho], [rho, 1]]

    # generate the samples
    sample1 = multivariate_normal(mu1, sigma).rvs(n)
    sample2 = multivariate_normal(mu2, sigma).rvs(n)

    # label the samples and extract the features and labels
    df1 = pd.DataFrame(sample1)
    df1['label'] = 1

    df2 = pd.DataFrame(sample2)
    df2['label'] = 2

    df = pd.concat([df1, df2], ignore_index=True)

    X = df.drop(columns='label')
    y = df.label
    
    # run the model
    model = LogisticRegression(solver='lbfgs').fit(X, y)
    y_pred_prob = model.predict_proba(X)[:,1]

    # compute AUC
    auc = roc_auc_score(y, y_pred_prob)
    return auc

Now we can sweep a range of values for `rho`.

In [ ]:
figsize(11, 5)
res = [(rho, multivariate_normal_auc(d=1, rho=rho))
       for rho in np.linspace(-0.9, 0.9)]

rhos, aucs = np.transpose(res)
plt.plot(rhos, aucs)
plt.xlabel('Correlation (rho)')
plt.ylabel('Area under ROC')
plt.title('AUC as a function of correlation');

### AUC as a function of d

The following function contains the code from the previous section, generalized to handle more than 2 dimensions.

In [ ]:
def multivariate_normal_auc(d, num_dims=2):
    """Generate multivariate normal samples and classify them.
    
    d: Cohen's effect size along each dimension
    num_dims: number of dimensions
    
    returns: AUC
    """
    # compute the mus
    mu1 = np.zeros(num_dims)
    mu2 = np.full(num_dims, d)

    # and sigma
    sigma = np.identity(num_dims)

    # generate the samples
    sample1 = multivariate_normal(mu1, sigma).rvs(n)
    sample2 = multivariate_normal(mu2, sigma).rvs(n)

    # label the samples and extract the features and labels
    df1 = pd.DataFrame(sample1)
    df1['label'] = 1

    df2 = pd.DataFrame(sample2)
    df2['label'] = 2

    df = pd.concat([df1, df2], ignore_index=True)

    X = df.drop(columns='label')
    y = df.label
    
    # run the model
    model = LogisticRegression(solver='lbfgs').fit(X, y)
    y_pred_prob = model.predict_proba(X)[:,1]

    # compute AUC
    auc = roc_auc_score(y, y_pred_prob)
    return auc

Confirming what we have seen before:

In [ ]:
multivariate_normal_auc(d=1, num_dims=1)

In [ ]:
multivariate_normal_auc(d=1, num_dims=2)

Now we can sweep a range of effect sizes.

In [ ]:
def compute_auc_vs_d(num_dims):
    """Sweep a range of effect sizes and compute AUC.
    
    num_dims: number of dimensions
    
    returns: list of 
    """
    effect_sizes = np.linspace(0, 4)

    return [(d, multivariate_normal_auc(d, num_dims))
            for d in effect_sizes]

In [ ]:
res1 = compute_auc_vs_d(1)
res2 = compute_auc_vs_d(2)
res3 = compute_auc_vs_d(3)
res4 = compute_auc_vs_d(4);

And plot the results.

In [ ]:
plot_auc_vs_d(res4, 'num_dim=4')
plot_auc_vs_d(res3, 'num_dim=3')
plot_auc_vs_d(res2, 'num_dim=2')
plot_auc_vs_d(res1, 'num_dim=1')
plt.title('AUC vs d for different numbers of features')
plt.legend();

With more features, the AUC gets better, assuming the features are independent.